In [23]:
images1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
images2 = images1[::-1]
image_pairs = [(image1, image2)
                   for image1 in images1 for image2 in images2]
print(10000*10000)

100000000


In [18]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity

def compare_images(image1, image2):
    # Load the images
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Convert images to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Compute the structural similarity index
    similarity = structural_similarity(gray1, gray2)

    return similarity

def find_similar_images(folder1, folder2):
    # Get the list of images in folder1 and folder2
    images1 = os.listdir(folder1)
    images2 = os.listdir(folder2)

    similar_images = []

    for image1 in images1:
        for image2 in images2:
            print("working")
            image_path1 = os.path.join(folder1, image1)
            image_path2 = os.path.join(folder2, image2)

            similarity = compare_images(image_path1, image_path2)

            if similarity > 0.9:  # Adjust this threshold as needed
                similar_images.append((image1, image2))
                print(f"Imagem A: {image1}   |   Imagem B: {image2}")
                exit()

    return similar_images

# # Example usage
# folder1 = './raw/10k/val'
# folder2 = 'raw/100k/val'

# similar_images = find_similar_images(folder1, folder2)

# for image_pair in similar_images:
#     print(f"Similar images: {image_pair[0]} and {image_pair[1]}")

In [12]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity
from multiprocessing import Pool

def compare_images(args):
    image1, image2 = args
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    similarity = structural_similarity(gray1, gray2)
    print(similarity)
    return image1, image2, similarity

def find_similar_images(folder1, folder2):
    images1 = os.listdir(folder1)
    images2 = os.listdir(folder2)

    image_pairs = [(os.path.join(folder1, image1), os.path.join(folder2, image2))
                   for image1 in images1 for image2 in images2]

    with Pool() as p:
        results = p.map(compare_images, image_pairs)

    similar_images = [(image1, image2) for image1, image2, similarity in results if similarity > 0.9]

    return similar_images

In [19]:
import os
import cv2
from PIL import Image
import imagehash

def compare_images(image1, image2):
    hash1 = imagehash.average_hash(Image.open(image1))
    hash2 = imagehash.average_hash(Image.open(image2))

    similarity = 1.0 - (hash1 - hash2) / len(hash1.hash)  # Compute similarity between hashes
    print(similarity)
    return similarity


In [50]:
import os
from PIL import Image
import imagehash
from tqdm import tqdm

def compare_images(image1, image2, threshold):
    hash1 = imagehash.average_hash(Image.open(image1))
    hash2 = imagehash.average_hash(Image.open(image2))

    # Compute the Hamming distance between the hashes
    distance = hash1 - hash2

    # Determine if the images are similar based on the threshold
    similar = distance <= threshold

    return similar

def find_similar_images(folder1, folder2, threshold=1):
    images1 = os.listdir(folder1)
    images2 = os.listdir(folder2)

    similar_images = []
    counter = 0
    for image1 in images1:
        counter +=1
        print(f"Imagem nº{counter}")
        for image2 in tqdm(images2):
            image_path1 = os.path.join(folder1, image1)
            image_path2 = os.path.join(folder2, image2)

            if compare_images(image_path1, image_path2, threshold):
                similar_images.append((image1, image2))
                print(f"Imagem A: {image1}   |   Imagem B: {image2}")

    return similar_images

In [10]:
import os
from PIL import Image
import imagehash
from tqdm import tqdm
from multiprocessing import Pool

def compare_images(args):
    image1, image2, threshold = args
    hash1 = imagehash.average_hash(Image.open(image1))
    hash2 = imagehash.average_hash(Image.open(image2))

    # Compute the Hamming distance between the hashes
    distance = abs(hash1 - hash2)

    # Determine if the images are similar based on the threshold
    similar = distance <= threshold
    
    if similar:
        print(f"Imagem A: {image1}   |   Imagem B: {image2}")
        return image1, image2

def find_similar_images(folder1, folder2, threshold=0):
    images1 = os.listdir(folder1)
    images2 = os.listdir(folder2)
    # print(f"Len 1: {len(images1)}  |  Len 1: {len(images2)}")
    similar_images = []
    image_pairs = [(os.path.join(folder1, image1), os.path.join(folder2, image2), threshold)
                   for image1 in images1 for image2 in images2]
    # print(len(image_pairs))
    # exit()
    with Pool() as p:
        results = p.map(compare_images, tqdm(image_pairs))

    return results

In [51]:
import pickle
# Example usage
folder1 = './raw/10k/val'
folder2 = 'raw/100k/val'
# folder1 = './abs'
# folder2 = './bsb'

similar_images = find_similar_images(folder1, folder2)
with open('potenciais.pickle', 'wb') as handle:
    pickle.dump(similar_images, handle, protocol=pickle.HIGHEST_PROTOCOL)
for image_pair in similar_images:
    print(f"Similar images: {image_pair[0]} and {image_pair[1]}")

Imagem nº1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:34<00:00, 105.77it/s]


Imagem nº2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:48<00:00, 92.02it/s]


Imagem nº3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:39<00:00, 100.63it/s]


Imagem nº4


 63%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                              | 6275/10000 [01:00<00:36, 103.46it/s]


KeyboardInterrupt: 

In [13]:
import pickle
# Example usage
folder1 = './raw/10k/val'
folder2 = 'raw/100k/val'
# folder1 = './abs'
# folder2 = './bsb'

similar_images = find_similar_images(folder1, folder2)
print(similar_images)
with open('potenciais.pickle', 'wb') as handle:
    pickle.dump(similar_images, handle, protocol=pickle.HIGHEST_PROTOCOL)
for image_pair in similar_images:
    try:
        print(f"Similar images: {image_pair[0]} and {image_pair[1]}")
    except:
        pass

 26%|█████████████████████████████████████████▋                                                                                                                     | 2618317/10000000 [00:21<00:13, 528337.68it/s]

Imagem A: ./raw/10k/val/a91b7555-00001280.jpg   |   Imagem B: raw/100k/val/ca2d32b3-5e8835ac.jpg
Imagem A: ./raw/10k/val/a4215cd6-00000000.jpg   |   Imagem B: raw/100k/val/b9d34460-5d3c8bcf.jpg
Imagem A: ./raw/10k/val/93fcc01f-00000000.jpg   |   Imagem B: raw/100k/val/b495e3ab-90180a0e.jpg
Imagem A: ./raw/10k/val/93fcc01f-00000000.jpg   |   Imagem B: raw/100k/val/bd9dbebc-fc042836.jpg
Imagem A: ./raw/10k/val/93fcc01f-00000000.jpg   |   Imagem B: raw/100k/val/c7729bd7-e38ccf62.jpg
Imagem A: ./raw/10k/val/a91b7555-00000365.jpg   |   Imagem B: raw/100k/val/bd5a0062-f462f63f.jpg


 32%|███████████████████████████████████████████████████▏                                                                                                          | 3236606/10000000 [1:15:43<2:43:20, 690.08it/s]

Imagem A: ./raw/10k/val/8a8a572f-261928a7.jpg   |   Imagem B: raw/100k/val/c95fecc3-41401a5f.jpg


 51%|████████████████████████████████████████████████████████████████████████████████▊                                                                             | 5111606/10000000 [1:20:41<07:18, 11142.13it/s]

...

Imagem A: ./raw/10k/val/a91b7555-00000885.jpg   |   Imagem B: raw/100k/val/c385a199-0bb19524.jpg


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [3:59:39<00:00, 695.42it/s]


Imagem A: ./raw/10k/val/a91b7555-00000370.jpg   |   Imagem B: raw/100k/val/bd5a0062-f462f63f.jpg
Imagem A: ./raw/10k/val/88f82b2a-93448a4d.jpg   |   Imagem B: raw/100k/val/bfe587eb-6335f4b3.jpg
Imagem A: ./raw/10k/val/88f82b2a-93448a4d.jpg   |   Imagem B: raw/100k/val/b4d0e72d-3b208072.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/ca2bcf0e-69e964d2.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/b1e9ee0e-67e26f2e.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/b7e8932c-3f90dbfc.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/bd1e9944-4fd7ac70.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/baa255c5-5c99a79a.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/c18df454-acb6d4fa.jpg
Imagem A: ./raw/10k/val/9d2cd320-407bfe7c.jpg   |   Imagem B: raw/100k/val/be791a81-a6b53fef.jpg


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Similar images: ./raw/10k/val/a91b7555-00001280.jpg and raw/100k/val/ca2d32b3-5e8835ac.jpg
Similar images: ./raw/10k/val/93fcc01f-00000000.jpg and raw/100k/val/b495e3ab-90180a0e.jpg
Similar images: ./raw/10k/val/93fcc01f-00000000.jpg and raw/100k/val/bd9dbebc-fc042836.jpg
Similar images: ./raw/10k/val/93fcc01f-00000000.jpg and raw/100k/val/c7729bd7-e38ccf62.jpg
Similar images: ./raw/10k/val/a91b7555-00000365.jpg and raw/100k/val/bd5a0062-f462f63f.jpg
Similar images: ./raw/10k/val/8a8a572f-261928a7.jpg and raw/100k/val/c95fecc3-41401a5f.jpg
Similar images: ./raw/10k/val/9f94fbbf-00000000.jpg and raw/100k/val/c90b5398-8d49f2f2.jpg
Similar images: ./raw/10k/val/9f94fbbf-00000000.jpg and raw/100k/val/b9102500-70bcb1c3.jpg
Similar images: ./raw/10k/val/9f94fbbf-00000000.jpg and raw/100k/val/c90a2111-6d28bf04.jpg
Similar images: ./raw/10k/val/9f94fbbf-00000000.jpg and raw/100k/val/bd1c6dd5-1534269e.jpg
Similar images: ./raw/10k/val/9f94fbbf-00000000.jpg and raw/100k/val/c3e986e8-1d7a6ddc.jpg

In [14]:
new_images = [x for x in similar_images if x is not None]

In [15]:
new_images

[('./raw/10k/val/a4215cd6-00000000.jpg', 'raw/100k/val/b9d34460-5d3c8bcf.jpg'),
 ('./raw/10k/val/a91b7555-00001280.jpg', 'raw/100k/val/ca2d32b3-5e8835ac.jpg'),
 ('./raw/10k/val/93fcc01f-00000000.jpg', 'raw/100k/val/b495e3ab-90180a0e.jpg'),
 ('./raw/10k/val/93fcc01f-00000000.jpg', 'raw/100k/val/bd9dbebc-fc042836.jpg'),
 ('./raw/10k/val/93fcc01f-00000000.jpg', 'raw/100k/val/c7729bd7-e38ccf62.jpg'),
 ('./raw/10k/val/a91b7555-00000365.jpg', 'raw/100k/val/bd5a0062-f462f63f.jpg'),
 ('./raw/10k/val/8a8a572f-261928a7.jpg', 'raw/100k/val/c95fecc3-41401a5f.jpg'),
 ('./raw/10k/val/9f94fbbf-00000000.jpg', 'raw/100k/val/c90b5398-8d49f2f2.jpg'),
 ('./raw/10k/val/9f94fbbf-00000000.jpg', 'raw/100k/val/b9102500-70bcb1c3.jpg'),
 ('./raw/10k/val/9f94fbbf-00000000.jpg', 'raw/100k/val/c90a2111-6d28bf04.jpg'),
 ('./raw/10k/val/9f94fbbf-00000000.jpg', 'raw/100k/val/bd1c6dd5-1534269e.jpg'),
 ('./raw/10k/val/9f94fbbf-00000000.jpg', 'raw/100k/val/c3e986e8-1d7a6ddc.jpg'),
 ('./raw/10k/val/86e4609e-00000000.jpg',

In [20]:
--ServerApp.iopub_data_rate_limit

NameError: name 'ServerApp' is not defined

In [21]:
with open('final.pickle', 'wb') as handle:
    pickle.dump(new_images, handle, protocol=pickle.HIGHEST_PROTOCOL)